# Support Vector Machine Regression Solution
### Imports

In [1]:
import h5py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
import sklearn as skl
from matplotlib import pyplot as plt
from time import time
#!pip install shap
import shap

### Loading & Preprocessing Data

In [2]:
def load_data(name):
    with h5py.File(f'{name}.h5', 'r') as f:
        return pd.DataFrame(f[name][:])

train = load_data('train')
test  = load_data('test')
print (f'Shape of training data set: {train.shape}')
print (f'Shape of test data set: {test.shape}')
all_variables = ['actualInteractionsPerCrossing', 'averageInteractionsPerCrossing', 'correctedActualMu', 'correctedAverageMu', 'correctedScaledActualMu', 'correctedScaledAverageMu', 'NvtxReco', 'p_nTracks', 'p_pt_track', 'p_eta', 'p_phi', 'p_charge', 'p_qOverP', 'p_z0', 'p_d0', 'p_sigmad0', 'p_d0Sig', 'p_EptRatio', 'p_dPOverP', 'p_z0theta', 'p_etaCluster', 'p_phiCluster', 'p_eCluster', 'p_rawEtaCluster', 'p_rawPhiCluster', 'p_rawECluster', 'p_eClusterLr0', 'p_eClusterLr1', 'p_eClusterLr2', 'p_eClusterLr3', 'p_etaClusterLr1', 'p_etaClusterLr2', 'p_phiClusterLr2', 'p_eAccCluster', 'p_f0Cluster', 'p_etaCalo', 'p_phiCalo', 'p_eTileGap3Cluster', 'p_cellIndexCluster', 'p_phiModCalo', 'p_etaModCalo', 'p_dPhiTH3', 'p_R12', 'p_fTG3', 'p_weta2', 'p_Reta', 'p_Rphi', 'p_Eratio', 'p_f1', 'p_f3', 'p_Rhad', 'p_Rhad1', 'p_deltaEta1', 'p_deltaPhiRescaled2', 'p_TRTPID', 'p_TRTTrackOccupancy', 'p_numberOfInnermostPixelHits', 'p_numberOfPixelHits', 'p_numberOfSCTHits', 'p_numberOfTRTHits', 'p_numberOfTRTXenonHits', 'p_chi2', 'p_ndof', 'p_SharedMuonTrack', 'p_E7x7_Lr2', 'p_E7x7_Lr3', 'p_E_Lr0_HiG', 'p_E_Lr0_LowG', 'p_E_Lr0_MedG', 'p_E_Lr1_HiG', 'p_E_Lr1_LowG', 'p_E_Lr1_MedG', 'p_E_Lr2_HiG', 'p_E_Lr2_LowG', 'p_E_Lr2_MedG', 'p_E_Lr3_HiG', 'p_E_Lr3_LowG', 'p_E_Lr3_MedG', 'p_ambiguityType', 'p_asy1', 'p_author', 'p_barys1', 'p_core57cellsEnergyCorrection', 'p_deltaEta0', 'p_deltaEta2', 'p_deltaEta3', 'p_deltaPhi0', 'p_deltaPhi1', 'p_deltaPhi2', 'p_deltaPhi3', 'p_deltaPhiFromLastMeasurement', 'p_deltaPhiRescaled0', 'p_deltaPhiRescaled1', 'p_deltaPhiRescaled3', 'p_e1152', 'p_e132', 'p_e235', 'p_e255', 'p_e2ts1', 'p_ecore', 'p_emins1', 'p_etconeCorrBitset', 'p_ethad', 'p_ethad1', 'p_f1core', 'p_f3core', 'p_maxEcell_energy', 'p_maxEcell_gain', 'p_maxEcell_time', 'p_maxEcell_x', 'p_maxEcell_y', 'p_maxEcell_z', 'p_nCells_Lr0_HiG', 'p_nCells_Lr0_LowG', 'p_nCells_Lr0_MedG', 'p_nCells_Lr1_HiG', 'p_nCells_Lr1_LowG', 'p_nCells_Lr1_MedG', 'p_nCells_Lr2_HiG', 'p_nCells_Lr2_LowG', 'p_nCells_Lr2_MedG', 'p_nCells_Lr3_HiG', 'p_nCells_Lr3_LowG', 'p_nCells_Lr3_MedG', 'p_pos', 'p_pos7', 'p_poscs1', 'p_poscs2', 'p_ptconeCorrBitset', 'p_ptconecoreTrackPtrCorrection', 'p_r33over37allcalo', 'p_topoetconeCorrBitset', 'p_topoetconecoreConeEnergyCorrection', 'p_topoetconecoreConeSCEnergyCorrection', 'p_weta1', 'p_widths1', 'p_widths2', 'p_wtots1', 'p_e233', 'p_e237', 'p_e277', 'p_e2tsts1', 'p_ehad1', 'p_emaxs1', 'p_fracs1', 'p_DeltaE', 'p_E3x5_Lr0', 'p_E3x5_Lr1', 'p_E3x5_Lr2', 'p_E3x5_Lr3', 'p_E5x7_Lr0', 'p_E5x7_Lr1', 'p_E5x7_Lr2', 'p_E5x7_Lr3', 'p_E7x11_Lr0', 'p_E7x11_Lr1', 'p_E7x11_Lr2', 'p_E7x11_Lr3', 'p_E7x7_Lr0', 'p_E7x7_Lr1' ]


Shape of training data set: (162500, 166)
Shape of test data set: (160651, 164)


In [3]:
def find_unique(X):
  means = np.mean(np.array(X), axis=0)
  unique_values, unique_indices = np.unique(means, return_index=True)
  sorted_indices = np.sort(unique_indices)
  unique_variables = []
  for i in unique_indices:
    unique_variables.append(all_variables[i])
  return unique_variables
def find_multivalues(X, unique_variables):
    multivalue_variables = []
    uniques = []
    j = 0
    for i in unique_variables:
        uniques.append(np.unique(X[i]))
        if len(uniques[j]) >= 2:
            multivalue_variables.append(i)
        j+=1
    return multivalue_variables

In [4]:
electron_indices = train['Truth'][train['Truth'][:]==True].index.values
X = train[all_variables].loc[electron_indices, all_variables]
print(X.shape)
unique_variables = find_unique(X)
X = X[unique_variables]
multivariables = find_multivalues(X, unique_variables)
X = X[multivariables]
X = pd.DataFrame(StandardScaler().fit_transform(X))
X.columns = multivariables
print('Found {0:d} unusuable features!'.format(len(all_variables)-len(multivariables)))
y = train['p_truth_E'].loc[electron_indices]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=16)


(121495, 160)
Found 19 unusuable features!


### Running generic Linear SVR with all features to select best ones according to feature weights

In [ ]:
# Build the SVM model
start = time()
svm = LinearSVR(dual=False, loss='squared_epsilon_insensitive')
svm.fit(X_train,y_train)
print('SVM takes {0:3.1f} minutes.'.format((time()-start)/60))

In [ ]:
def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names), reverse=True))
    plt.barh(range(10), imp[:10], align='center')
    plt.yticks(range(10), names[:10])
    plt.show()
    return names[:10]
best_features = f_importances(svm.coef_, multivariables)

In [ ]:
X_best = X[list(best_features)]
X_best.columns = list(best_features)
X_btrain, X_btest, y_train, y_test = train_test_split(X_best, y, test_size=.25, random_state=16)

### Running SVM with 10 best features

In [26]:
svm = LinearSVR(dual=False, loss='squared_epsilon_insensitive')
svm.fit(X_btrain, y_train)
energies = svm.predict(X_btest)

In [33]:
print(skl.metrics.explained_variance_score(y_test.values, energies))

0.9111395020800495


In [31]:
X_testing = test[list(best_features)]
X_testing = pd.DataFrame(StandardScaler().fit_transform(X_testing))
X_testing.columns = list(best_features)
dtest_predictions = svm.predict(X_testing)

In [32]:
pd.Series(dtest_predictions).to_csv('Regression_LaurentLindpointner_SVM.txt', header=False)
pd.Series(list(best_features)).to_csv('Regression_LaurentLindpointner_SVM_VariableList.txt', index=False, header=False)